# Three Dimensional Cubes 

<section class="objectives panel panel-warning">
<div class="panel-heading">
<h3><span class="fa fa-certificate"></span> Learning Objectives </h3>
</div>
<ul>
    <li>Plotting cubes with WCS objects</li>
    <li>Exploring Cubes with SunPy</li>
</ul>
</section>

Cube arrays are quite common in ground based and spectral type data, so we are going to use one as an example. SunPy has no wrapper class (like `Map`) for multi-dimensional data so this is an excellent way to use the things we have covered so far.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS

import sunpy.coordinates

Download the file:

In [ ]:
from astropy.utils.data import download_file

cube_file = download_file("http://data.sunpy.org/CRISP_LXY_Cube.fits.gz", cache=True)

In [ ]:
cube_file = "./CRISP_LXY_Cube.fits.gz"

Read in the cube FITS file

In [ ]:
hdulist = fits.open(cube_file)
hdulist.info()

In [ ]:
header = hdulist[0].header
data = hdulist[0].data

In [ ]:
data.shape

From this cube file we can construct an [`astropy.wcs`](http://docs.astropy.org/en/stable/wcs/index.html) object from the FITS header and then plot images from the array.

You can build a `WCS` object from a filepath or a header or a dictionary. Here we use the header object from the cube.

In [ ]:
wcs = WCS(header)

For this `WCS` object to correctly create SunPy coordinates objects, it needs to have correct Sun-Earth distance information. By default it currently uses 1 AU, which is almost always incorrect!

We can do this from the date of the observations using `sunpy.sun`. SunPy will attempt to read the observer location information from three attributes. The one we are setting is `dsun`.

In [ ]:
from sunpy import sun

# Set the correct Sun-Earth distance
wcs.dsun = sun.sunearth_distance(wcs.wcs.dateobs).to(u.m)

In [ ]:
wcs

This cube has three axes: Wavelength, Longitude, Latitude. Note that the WCS axis ordering is out of order with the array, this is due to the 

In [ ]:
data.shape

**Note:** The WCS axis ordering and the array order are reversed. The wavelength axis here is 32 long.

### Plot a Line Wing

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=wcs, slices=('x', 'y', 0))

ax.imshow(data[0, :, :], origin='lower', interpolation='none', cmap='gray')

### Plot the Line Core

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=wcs, slices=('x', 'y', 18))

ax.imshow(data[18, :, :], origin='lower', interpolation='none', cmap='gray')

lon, lat, wave = ax.coords

lat.set_major_formatter('s.s')
lon.set_major_formatter('s.s')

### Plot a Wavelength - Latitude Slice

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=wcs, slices=('y', 330, 'x'))

# Transpose the array so the wavelength dimension is last ('x')
ax.imshow(data[:, 330, :].T, origin='lower', interpolation='none', cmap='gray', aspect='auto')

lon, lat, wave = ax.coords

# Make the wavelength ticks nice
wave.set_major_formatter('x.xx')
wave.set_format_unit(u.nm)
wave.set_ticks(spacing=0.05*u.nm)

lat.set_major_formatter('s.s')
lon.set_major_formatter('s.s')

lat.set_axislabel("Helioprojective Latitude [arcsec]")
lon.set_axislabel("Helioprojective Longitude [arcsec]")
wave.set_axislabel("Wavelength [nm]")

### Overplot the Heliograpic Grid

Extract a spatial axis only WCS

In [ ]:
sub = wcs.sub(axes=[1,2])
sub

In [ ]:
fig = plt.figure()
ax = plt.subplot(projection=sub)
ax.imshow(data[0], cmap='gray', interpolation='none', origin='lower')
lon, lat = ax.coords
lon.set_major_formatter('s.s')
lat.set_major_formatter('s.s')

lat.set_axislabel("Helioprojective Latitude [arcsec]")
lon.set_axislabel("Helioprojective Longitude [arcsec]")

lon.set_ticks_position('bl')
lat.set_ticks_position('bl')

overlay = ax.get_coords_overlay('heliographic_stonyhurst')
overlay.grid()

hglon, hglat = overlay

hglon.set_ticks_position('tr')
hglat.set_ticks_position('tr')


hglat.set_axislabel("Heliographic Latitude [deg]")
hglon.set_axislabel("Heliographic Longitude [deg]")

## Cube Exploration with SunPy

In [ ]:
from sunpy.visualization.imageanimator import ImageAnimator

In [ ]:
ImageAnimator(data, cmap='gray')